<a href="https://colab.research.google.com/github/AmbroseNTK/ClassroomEvaluation/blob/master/notebooks/Student_Behaviors_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Student's behaviors Detection
Student's activities in class in one of the important critical to evaluate course's quality.
Based on Object Detection problem, using TensorFlow Object Detection API to make a model.

First of all, I need prepare training data using LabelImg tools in other to crop student's behaviors in classroom video recording.

Then, I upload it to my Google Drive to help training process on Colab. Why I use Colab? Because Colab allow me use GPU Tesla K80 which was optimized for the Machine Learning tasks, specially, TensorFlow.

## 1. Data Preparation
Mount Google drive to this kernel in other to copy training data

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Download TensorFlow models from GitHub

In [0]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 28196 (delta 0), reused 0 (delta 0), pack-reused 28195
Receiving objects: 100% (28196/28196), 509.34 MiB | 29.59 MiB/s, done.
Resolving deltas: 100% (17368/17368), done.
Checking out files: 100% (3004/3004), done.


Copy and unzip my dataset

In [0]:
!cp "/content/drive/My Drive/dataset.zip" models/research/object_detection

In [0]:
!cd models/research/object_detection && unzip dataset.zip 

Archive:  dataset.zip
   creating: dataset/
   creating: dataset/test/
  inflating: dataset/test/52M20S_1559289140_840.jpg  
  inflating: dataset/test/52M20S_1559289140_840.xml  
  inflating: dataset/test/52M20S_1559289140_960.jpg  
  inflating: dataset/test/52M20S_1559289140_960.xml  
  inflating: dataset/test/52M20S_1559292740_0.jpg  
  inflating: dataset/test/52M20S_1559292740_0.xml  
  inflating: dataset/test/52M20S_1559292740_1080.jpg  
  inflating: dataset/test/52M20S_1559292740_1080.xml  
  inflating: dataset/test/52M20S_1559292740_120.jpg  
  inflating: dataset/test/52M20S_1559292740_120.xml  
  inflating: dataset/test/52M20S_1559292740_240.jpg  
  inflating: dataset/test/52M20S_1559292740_240.xml  
  inflating: dataset/test/52M20S_1559292740_360.jpg  
  inflating: dataset/test/52M20S_1559292740_360.xml  
  inflating: dataset/test/52M20S_1559292740_480.jpg  
  inflating: dataset/test/52M20S_1559292740_480.xml  
  inflating: dataset/test/52M20S_1559292740_600.jpg  
  inflating: 

## 2. Setup API

Download pretrained object detection model from TensorFlow

In this case, I use Inception v2.

In [0]:
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

--2019-07-08 04:05:16--  http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.76.128, 2a00:1450:400c:c00::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.76.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149119618 (142M) [application/x-tar]
Saving to: ‘faster_rcnn_inception_v2_coco_2018_01_28.tar.gz’

faster_rcnn_incepti 100%[===================>] 142.21M  89.2MB/s    in 1.6s    

2019-07-08 04:05:18 (89.2 MB/s) - ‘faster_rcnn_inception_v2_coco_2018_01_28.tar.gz’ saved [149119618/149119618]



In [0]:
!tar -xvzf faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

faster_rcnn_inception_v2_coco_2018_01_28/
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.index
faster_rcnn_inception_v2_coco_2018_01_28/checkpoint
faster_rcnn_inception_v2_coco_2018_01_28/pipeline.config
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.meta
faster_rcnn_inception_v2_coco_2018_01_28/saved_model/
faster_rcnn_inception_v2_coco_2018_01_28/saved_model/saved_model.pb
faster_rcnn_inception_v2_coco_2018_01_28/saved_model/variables/
faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb


In [0]:
!mkdir models/research/object_detection/training

In [0]:
!cp -r faster_rcnn_inception_v2_coco_2018_01_28 models/research/object_detection/faster_rcnn_inception_v2_coco_2018_01_28

Install TensorFlow Object Detection API for Python 3.6

In [0]:
!cd models/research && protoc object_detection/protos/*.proto --python_out=.

In [0]:
!cd models/research/ && python setup.py build && python setup.py install && export PYTHONPATH=$PYTHONPATH:pwd:pwd/slim

running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/export_inference_graph.py -> build/lib/object_detection
copying object_detection/model_tpu_main.py -> build/lib/object_detection
copying object_detection/eval_util_test.py -> build/lib/object_detection
copying object_detection/model_lib_test.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph.py -> build/lib/object_detection
copying object_detection/inputs.py -> build/lib/object_detection
copying object_detection/exporter.py -> build/lib/object_detection
copying object_detection/__init__.py -> build/lib/object_detection
copying object_detection/model_lib.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib.py -> build/lib/object_detection
copying object_detection/inputs_test.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib_test.py -> build/lib/object_detection
cop

In [0]:
!cd models/research && export PYTHONPATH=$PYTHONPATH:pwd:pwd/slim && cp -r ./slim/nets ./object_detection

In [0]:
!cp -r models/research/slim/* models/research/object_detection/

## 3. Training
Start training using script **model_main.py**

In training process, please notice the location of checkpoint. It is put in **/tmp/\<ID\>/** folder, where ID is generated in the run time 

In [0]:
!cd models/research/object_detection/ && python model_main.py --logtostderr --train_dir=training/ --pipeline_config_path=training/frcnn-colab.config

W0708 04:14:31.840249 139747274839936 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0708 04:14:31.872673 139747274839936 deprecation_wrapper.py:119] From /content/models/research/object_detection/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0708 04:14:31.886577 139747274839936 deprecation_wrapper.py:119] From /content/models/research/object_detection/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0708 04:14:31.900410 139747274839936 deprecation_wrapper.py:119] From model_main.py:109: The name tf.app.run is deprecated. Ple

In this case, ID is **tmpaiqr9_b9**

Copy all file from this folder and zip them into **checkpoints.zip**

This checkpoints can be used for pretraining

In [0]:
!cp -r /tmp/tmpaiqr9_b9 /content/

In [0]:
!zip -r checkpoints.zip tmpaiqr9_b9

  adding: tmpaiqr9_b9/ (stored 0%)
  adding: tmpaiqr9_b9/eval_0/ (stored 0%)
  adding: tmpaiqr9_b9/eval_0/events.out.tfevents.1562560099.83ff4eb6abd7 (deflated 8%)
  adding: tmpaiqr9_b9/graph.pbtxt (deflated 97%)
  adding: tmpaiqr9_b9/checkpoint (deflated 71%)
  adding: tmpaiqr9_b9/model.ckpt-23477.meta (deflated 94%)
  adding: tmpaiqr9_b9/model.ckpt-21067.data-00000-of-00001 (deflated 8%)
  adding: tmpaiqr9_b9/model.ckpt-22284.data-00000-of-00001 (deflated 8%)
  adding: tmpaiqr9_b9/model.ckpt-23477.data-00000-of-00001 (deflated 8%)
  adding: tmpaiqr9_b9/model.ckpt-19845.index (deflated 72%)
  adding: tmpaiqr9_b9/model.ckpt-19845.meta (deflated 94%)
  adding: tmpaiqr9_b9/model.ckpt-21067.index (deflated 72%)
  adding: tmpaiqr9_b9/model.ckpt-19845.data-00000-of-00001 (deflated 8%)
  adding: tmpaiqr9_b9/model.ckpt-18662.meta (deflated 94%)
  adding: tmpaiqr9_b9/model.ckpt-18662.index (deflated 72%)
  adding: tmpaiqr9_b9/model.ckpt-18662.data-00000-of-00001 (deflated 8%)
  adding: tmpaiqr

To deploy usable model, using **export_inference_graph.py** script

In [0]:
!cd models/research/object_detection/ && python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/frcnn-colab.config --trained_checkpoint_prefix /content/tmpaiqr9_b9/model.ckpt-23477 --output_directory /content/inference_graph

W0708 07:33:11.189002 139721312864128 deprecation_wrapper.py:119] From /content/models/research/object_detection/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0708 07:33:11.203261 139721312864128 deprecation_wrapper.py:119] From /content/models/research/object_detection/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0708 07:33:11.217183 139721312864128 deprecation_wrapper.py:119] From export_inference_graph.py:156: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0708 07:33:11.218054 139721312864128 deprecation_wrapper.py:119] From export_inference_graph.py:139: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0708 07:33:11.225621 139721312864128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/object_detection-0.1-py3.6.egg/object_detection/exporter.py:367: The name tf.gfil

Zip the result and copy it to GDrive

In [0]:
!cp -r /content/models/research/object_detection/content /content

In [0]:
!zip -r inferenced.zip /content/inference_graph

  adding: content/inference_graph/ (stored 0%)
  adding: content/inference_graph/checkpoint (deflated 42%)
  adding: content/inference_graph/model.ckpt.index (deflated 70%)
  adding: content/inference_graph/model.ckpt.meta (deflated 95%)
  adding: content/inference_graph/saved_model/ (stored 0%)
  adding: content/inference_graph/saved_model/variables/ (stored 0%)
  adding: content/inference_graph/saved_model/saved_model.pb (deflated 11%)
  adding: content/inference_graph/frozen_inference_graph.pb (deflated 11%)
  adding: content/inference_graph/model.ckpt.data-00000-of-00001 (deflated 7%)
  adding: content/inference_graph/pipeline.config (deflated 66%)


In [0]:
!cp inferenced.zip "/content/drive/My Drive"